In [23]:
from dotenv import find_dotenv, load_dotenv
from transformers import pipeline

from langchain.prompts import PromptTemplate

import requests
import os

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain



import tensorflow as tf
import torch as pt
# from transformers import TFGPT2LMHeadModel,GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM


In [3]:
load_dotenv(find_dotenv())


True

In [4]:
HUGGINGFACEHUB_API_TOKENS = os.getenv("api_token")


In [5]:
def img2text(url):
    image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
    text = image_to_text(url)[0]['generated_text']
    print(text)
    return text

In [26]:
image_url = 'black_shirt_my_pic-removebg-preview-PhotoRoom.png-PhotoRoom.png'
image_caption = img2text(image_url)

c:\Users\Amit\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a man with glasses and a black shirt


### Text summarization

#### GPT-2

In [7]:
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id = tokenizer.eos_token_id)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [7]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta",pad_token_id = tokenizer.eos_token_id)

c:\Users\Amit\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Amit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [12]:
# def generate_text(input):
#     input_ids = tokenizer.encode(input)
#     beam_output = model.generate(input_ids, max_length=100, num_beams=1, no_repeat_ngram_size=2, early_stopping=True)
#     output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     output = ' '.join(output.split())  # Remove extra spaces
#     return output


In [24]:
def generate_text(input):
    input_ids = tokenizer(input, return_tensors="pt")
    generate_ids = model.generate(input_ids.input_ids, max_length=30)
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    output = ' '.join(output.split())  # Remove extra spaces
    return output


In [28]:
input = image_caption
input = input.replace(",", "")
# print(input)
extend_text = generate_text(input)

In [31]:
def text2speech(message):
    API_URL = "https://api-inference.huggingface.co/models/microsoft/speecht5_tts"
    headers = {"Authorization": f"Bearer {HUGGINGFACEHUB_API_TOKENS}"}
    payloads = {
        "input": message
    }

    response = requests.post(API_URL, headers = headers,json = payloads)
    with open('audio.wav','wb') as file:
        file.write(response.content)

In [32]:
text2speech(extend_text)